<a href="https://colab.research.google.com/github/distinctkemi/CE888/blob/main/%5CLabs%5CLab%206%5CMy_Recomemnder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New section

##Data Description
* Ratings are real values ranging from -10.00 to +10.00 (the value "99" 
corresponds to "null" = "not rated").
One row per user
* The first column gives the number of jokes rated by that user. The next 100 columns give the ratings for jokes 01 - 100.
* The sub-matrix including only columns {5, 7, 8, 13, 15, 16, 17, 18, 19, 20} is dense. Almost all users have rated those jokes (see discussion of "universal queries" in the above paper).

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
np.set_printoptions(precision=3)

In [2]:
#Loading the data
data = pd.read_csv("https://raw.githubusercontent.com/albanda/CE888/master/lab4-recommender/jester-data-1.csv", header = None)
data

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,74,-7.82,8.79,-9.66,-8.16,-7.52,-8.50,-9.85,4.17,-8.98,...,2.82,99.00,99.00,99.00,99.00,99.00,-5.63,99.00,99.00,99.00
1,100,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88,...,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
2,49,99.00,99.00,99.00,99.00,9.03,9.27,9.03,9.27,99.00,...,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00
3,48,99.00,8.35,99.00,99.00,1.80,8.16,-2.82,6.21,99.00,...,99.00,99.00,99.00,0.53,99.00,99.00,99.00,99.00,99.00,99.00
4,91,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,-0.44,...,5.19,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24978,100,0.44,7.43,9.08,2.33,3.20,6.75,-8.79,-0.53,-8.74,...,8.83,-1.21,9.22,-6.70,8.45,9.03,6.55,8.69,8.79,7.43
24979,91,9.13,-8.16,8.59,9.08,0.87,-8.93,-3.50,5.78,-8.11,...,-1.17,-5.73,-1.46,0.24,9.22,-8.20,-7.23,-8.59,9.13,8.45
24980,39,99.00,99.00,99.00,99.00,-7.77,99.00,6.70,-6.75,99.00,...,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00
24981,37,99.00,99.00,99.00,99.00,-9.71,99.00,4.56,-8.30,99.00,...,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00


In [50]:
data.shape

(24983, 100)

In [3]:
#Dropping the first column based on description of data
data.drop(data.columns[[0]], axis=1, inplace = True)
data.head()


,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
0,-7.82,8.79,-9.66,-8.16,-7.52,-8.50,-9.85,4.17,-8.98,-4.76,...,2.82,99.00,99.00,99.00,99.00,99.00,-5.63,99.00,99.00,99.00
1,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88,9.22,...,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
2,99.00,99.00,99.00,99.00,9.03,9.27,9.03,9.27,99.00,99.00,...,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00
3,99.00,8.35,99.00,99.00,1.80,8.16,-2.82,6.21,99.00,1.84,...,99.00,99.00,99.00,0.53,99.00,99.00,99.00,99.00,99.00,99.00
4,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,-0.44,5.73,...,5.19,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60


In [4]:
#Replacing the 99 to nans
test_set = data.replace(99, np.NaN)
test_set

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
0,-7.82,8.79,-9.66,-8.16,-7.52,-8.50,-9.85,4.17,-8.98,-4.76,...,2.82,NaN,NaN,NaN,NaN,NaN,-5.63,NaN,NaN,NaN
1,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88,9.22,...,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
2,NaN,NaN,NaN,NaN,9.03,9.27,9.03,9.27,NaN,NaN,...,NaN,NaN,NaN,9.08,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,8.35,NaN,NaN,1.80,8.16,-2.82,6.21,NaN,1.84,...,NaN,NaN,NaN,0.53,NaN,NaN,NaN,NaN,NaN,NaN
4,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,-0.44,5.73,...,5.19,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24978,0.44,7.43,9.08,2.33,3.20,6.75,-8.79,-0.53,-8.74,7.23,...,8.83,-1.21,9.22,-6.70,8.45,9.03,6.55,8.69,8.79,7.43
24979,9.13,-8.16,8.59,9.08,0.87,-8.93,-3.50,5.78,-8.11,4.90,...,-1.17,-5.73,-1.46,0.24,9.22,-8.20,-7.23,-8.59,9.13,8.45
24980,NaN,NaN,NaN,NaN,-7.77,NaN,6.70,-6.75,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24981,NaN,NaN,NaN,NaN,-9.71,NaN,4.56,-8.30,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
data.describe(include = 'all')

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
count,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,...,24983.000000,24983.00000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000,24983.000000
mean,34.401435,27.094089,33.263717,36.244936,0.427236,19.342119,-0.401803,-0.616110,36.228421,19.520763,...,61.462019,60.27989,60.421701,59.059621,58.749897,58.541104,58.017246,57.061650,55.753357,60.138090
std,46.707059,44.228006,46.747535,48.812967,5.472657,37.860770,5.741616,5.008894,48.228364,38.329975,...,47.335405,47.94644,47.359847,48.196204,48.310728,48.130562,48.164681,48.717455,49.237139,47.913531
min,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,...,-9.950000,-9.95000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000
25%,-0.150000,-2.180000,-1.145000,-3.350000,-3.690000,-0.390000,-5.290000,-4.610000,-1.775000,-0.830000,...,4.760000,3.59000,4.610000,3.250000,3.160000,3.300000,3.590000,2.670000,1.335000,3.930000
50%,5.050000,3.690000,4.660000,3.450000,1.120000,3.540000,-0.050000,-0.290000,4.030000,3.400000,...,99.000000,99.00000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000
75%,99.000000,99.000000,99.000000,99.000000,4.470000,8.010000,3.930000,3.060000,99.000000,8.110000,...,99.000000,99.00000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000
max,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,...,99.000000,99.00000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000


In [6]:
test_set.describe(include = 'all')

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
count,16452.000000,18184.000000,16642.000000,15608.000000,24977.000000,20431.000000,24976.000000,24981.000000,15752.000000,20324.000000,...,9672.000000,9889.000000,9988.000000,10200.000000,10265.000000,10370.000000,10520.000000,10666.000000,10910.000000,9943.000000
mean,0.904573,0.208460,0.316574,-1.449114,0.403556,1.594447,-0.429662,-0.624085,-0.557031,1.301231,...,2.038527,1.179744,2.504041,1.173482,1.039130,1.528003,1.673750,0.767692,-0.031244,1.354711
std,5.171798,5.597327,5.399806,5.234654,5.255700,4.900540,5.495931,4.929144,5.162224,5.087359,...,5.060445,5.127291,4.611339,5.140307,5.167043,4.852069,5.025076,5.393833,5.176806,5.300409
min,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,...,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000
25%,-2.670000,-4.420000,-4.030000,-5.870000,-3.690000,-1.310000,-5.290000,-4.610000,-4.760000,-2.090000,...,-0.932500,-2.380000,-0.100000,-2.280000,-2.570000,-1.347500,-1.170000,-3.400000,-4.170000,-2.330000
50%,1.600000,0.830000,0.780000,-1.460000,1.120000,2.230000,-0.050000,-0.290000,-0.190000,1.940000,...,2.860000,1.800000,3.160000,1.750000,1.650000,2.140000,2.480000,1.460000,0.290000,2.140000
75%,4.900000,4.820000,4.660000,2.520000,4.470000,5.440000,3.880000,3.060000,3.300000,5.390000,...,6.120000,5.290000,6.070000,5.290000,5.100000,5.277500,5.630000,5.150000,3.980000,5.730000
max,9.370000,9.370000,9.370000,9.370000,9.370000,9.470000,9.610000,9.760000,9.900000,9.370000,...,9.560000,9.420000,9.370000,9.420000,9.370000,9.370000,9.370000,9.370000,9.370000,9.370000


In [7]:
#Finding the best- and the worst-rated jokes
average = test_set.mean(axis = 0)
average.sort_values(ascending = False)

50    3.665085
89    3.574720
36    3.312500
27    3.189821
32    3.155988
        ...   
13   -1.762882
57   -1.990664
44   -2.111891
16   -3.104818
58   -3.833880
Length: 100, dtype: float64

Labeling the label 10% of the dataset cells that are not NaNs as 99. 

In [8]:
arr = test_set.values
print(arr.shape)

(24983, 100)


In [9]:
rated = np.where(arr!=99)
print(len(rated[0]), rated[1].shape)

2498300 (2498300,)


In [10]:
def replace(orig, percentage=0.1):
  """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """
  new_data = orig.copy()
  rated = np.where(orig!= np.nan)
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

In [11]:
new_arr, idx = replace(arr, 0.1)

In [ ]:
arr[idx[0][0], idx[1][0]]

1.5

In [12]:
n_latent_factors = 2

user_ratings = data.values
# Initialise as random values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))

In [13]:
latent_item_features

array([[0.278, 0.94 ],
       [0.922, 0.341],
       [0.855, 0.84 ],
       [0.328, 0.583],
       [0.073, 0.577],
       [0.851, 0.549],
       [0.301, 0.523],
       [0.108, 0.977],
       [0.032, 0.863],
       [0.259, 0.302],
       [0.18 , 0.479],
       [0.144, 0.207],
       [0.162, 0.05 ],
       [0.301, 0.951],
       [0.14 , 0.238],
       [0.942, 0.799],
       [0.419, 0.055],
       [0.398, 0.256],
       [0.268, 0.299],
       [0.574, 0.751],
       [0.963, 0.149],
       [0.103, 0.669],
       [0.864, 0.009],
       [0.554, 0.778],
       [0.593, 0.445],
       [0.283, 0.919],
       [0.648, 0.271],
       [0.195, 0.331],
       [0.022, 0.992],
       [0.179, 0.441],
       [0.563, 0.547],
       [0.351, 0.874],
       [0.228, 0.802],
       [0.493, 0.355],
       [0.954, 0.174],
       [0.984, 0.359],
       [0.75 , 0.894],
       [0.34 , 0.792],
       [0.666, 0.977],
       [0.245, 0.697],
       [0.838, 0.671],
       [0.476, 0.959],
       [0.635, 0.703],
       [0.0

In [24]:
data.iloc[17273,26]

-6.84

In [51]:
#rating that user 20732 has given to joke 52
data.iloc[20732, 52]

5.73

In [14]:
latent_user_preferences

array([[0.725, 0.226],
       [0.992, 0.87 ],
       [0.911, 0.156],
       ...,
       [0.821, 0.547],
       [0.97 , 0.989],
       [0.521, 0.901]])

In [15]:
def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)


def train(user_id, item_id, rating, alpha=0.0001):
    #print(item_id)
    predicted_rating = predict_rating(user_id, item_id)
    err =  predicted_rating - rating
    #print(err)
    user_pref_values = latent_user_preferences[user_id]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    

def sgd(iterations):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    mse_history = []
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = user_ratings[user_id, item_id]
                if not np.isnan(rating) and rating != 99:
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        if (iteration % 10000) == 0:
            print('Iteration %d/%d:\tMSE=%.6f' % (iteration, iterations, mse))
            mse_history.append(mse)
    return mse_history

In [16]:
predictions = latent_user_preferences.dot(latent_item_features.T)
predictions

array([[0.414, 0.745, 0.81 , ..., 0.419, 0.436, 0.41 ],
       [1.093, 1.21 , 1.579, ..., 0.598, 0.631, 0.765],
       [0.4  , 0.893, 0.91 , ..., 0.521, 0.539, 0.469],
       ...,
       [0.742, 0.943, 1.162, ..., 0.487, 0.511, 0.57 ],
       [1.199, 1.231, 1.66 , ..., 0.591, 0.626, 0.798],
       [0.992, 0.787, 1.203, ..., 0.333, 0.359, 0.563]])

In [17]:
num_iter = 30
hist = sgd(num_iter)  # Note how the MSE decreases with the number of iterations
plt.figure()
plt.plot(np.arange(0, num_iter, 10000), hist)
plt.xlabel("Iterations")
plt.ylabel("MSE")
plt.show()

KeyboardInterrupt: ignored

In [19]:
df_movie = pd.read_excel('movies_latent_factors.xlsx', sheet_name= 0)
df_movie.head()

,Movie ID,Title,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
0,11,Star Wars: Episode IV - A New Hope (1977),-1.521848,-1.038507,2.027269,0.247933,-0.594548,2.513260,-1.848910,0.476710,-0.224146,-0.760681,-0.973915,0.862379,0.403861,1.129616,-0.248806
1,12,Finding Nemo (2003),-0.342185,-0.296586,-0.385962,2.443297,-1.097015,-0.619465,0.572887,0.329516,-0.712228,0.323554,-0.561948,-0.142405,-0.564415,0.506876,1.274993
2,13,Forrest Gump (1994),-2.240888,-0.438815,-2.275177,0.614548,0.904469,0.711919,-0.420876,1.302036,-0.868418,-0.160122,0.633667,0.133138,-0.330276,-2.209004,-0.419092
3,14,American Beauty (1999),-0.634531,2.186059,-0.066681,0.086197,0.517558,-0.185319,-0.412352,0.063841,0.075937,-0.577682,0.526803,-1.465557,-0.819682,0.549010,-0.681191
4,22,Pirates of the Caribbean: The Curse of the Bla...,0.517348,-1.456763,0.369161,0.073903,-0.098332,-0.594722,0.828888,0.033626,-0.707414,0.717877,1.394326,-0.786986,1.429191,0.855746,-2.292566


In [56]:
df_users = pd.read_excel('movies_latent_factors.xlsx', sheet_name= 1)
df_users.head()

,User,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
0,4768,-0.204024,0.161079,-0.090447,0.138495,-0.162934,0.163894,0.051502,-0.088582,0.126829,0.065967,0.085008,0.355404,0.007108,-0.118663,-0.039125
1,156,-0.189652,-0.178979,-0.091490,-0.000823,-0.032646,0.177209,-0.098123,-0.068283,-0.011575,0.120866,-0.009931,-0.048606,0.045916,0.113671,0.179873
2,5323,-0.115308,-0.090886,-0.053129,0.018472,-0.068081,-0.004828,0.113005,0.102107,0.034758,0.000693,-0.073712,-0.019460,0.108372,0.054471,-0.109552
3,174,-0.227462,-0.272532,-0.017231,0.054324,0.214755,-0.072639,-0.033122,-0.086508,-0.131479,0.180403,0.095890,-0.082396,0.036767,-0.165438,0.050692
4,4529,-0.014616,-0.102218,-0.107935,0.155784,-0.123362,-0.118228,-0.013549,-0.050622,0.058698,-0.159600,-0.142382,-0.132836,-0.039897,0.129063,0.102669


In [21]:

aa = df_movie[['Factor7','Movie ID']]
aa

,Factor7,Movie ID
0,-1.848910,11
1,0.572887,12
2,-0.420876,13
3,-0.412352,14
4,0.828888,22
...,...,...
95,0.639252,9806
96,-0.007971,10020
97,-0.037449,36657
98,-0.267268,36658


In [26]:
i=df_movie[df_movie['Movie ID']==280]
print(i)

    Movie ID                              Title   Factor1  Factor2   Factor3  \
34       280  Terminator 2: Judgment Day (1991) -0.614778 -0.85617  0.418031   

     Factor4   Factor5  Factor6   Factor7   Factor8   Factor9  Factor10  \
34 -0.979775 -1.730847  1.58805  0.704612  0.385626  1.719533 -0.512454   

    Factor11  Factor12  Factor13  Factor14  Factor15  
34  1.034302 -0.788705 -0.385154 -0.395198 -0.501914  


In [57]:
#what's the loading (i.e., value) of Factor11 for user with ID=156
j = df_users[df_users['User']==156]
j

,User,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
1,156,-0.189652,-0.178979,-0.09149,-0.000823,-0.032646,0.177209,-0.098123,-0.068283,-0.011575,0.120866,-0.009931,-0.048606,0.045916,0.113671,0.179873


In [62]:
#which of the following IDs corresponds to the movie with the lowest loading of factor 9?

bb = df_movie[['Movie ID','Factor9','Title']]
bb.sort_values('Factor9')

,Movie ID,Factor9,Title
51,604,-3.492897,The Matrix Reloaded (2003)
79,1894,-3.324583,Star Wars: Episode II - Attack of the Clones (...
66,809,-2.325862,Shrek 2 (2004)
65,808,-2.016609,Shrek (2001)
37,414,-1.720265,Batman Forever (1995)
...,...,...,...
9,85,1.438862,Raiders of the Lost Ark (Indiana Jones and the...
14,120,1.444931,The Lord of the Rings: The Fellowship of the R...
35,329,1.570226,Jurassic Park (1993)
34,280,1.719533,Terminator 2: Judgment Day (1991)


In [63]:
#Rating Predicitiom Function
from pandas.core.arrays.numeric import T
def predict_rating(user_id, movie_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = df_users[df_users["User"] == user_id]
    movie_preference = df_movie[df_movie["Movie ID"]  == movie_id]
    user_preference = user_preference.drop(user_preference.columns[[0]], axis = 1)
    movie_preference = movie_preference.drop(movie_preference.columns[[0,1]], axis = 1)
    user_preference = user_preference.to_numpy()
    movie_preference = movie_preference.to_numpy()

    return user_preference.dot(movie_preference.T)

In [64]:
#In the movies dataset, predict the rating that the user with ID 783 will give to the movie with ID 4327. Give your answer with two decimal values (e.g., 0.12)
predict_rating(783, 4327)

array([[-0.021]])

In [28]:
predict_rating(3878 ,8587)

array([[0.411]])

In [65]:

#In the movies dataset, predict the rating that the user with ID 4940 will give to the movie with ID 808. Give your answer with two decimal values (e.g., 0.12).
predict_rating(4940, 808)

array([[-0.066]])

In [30]:
from pandas.core.arrays.numeric import T
def predict_movies(user_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = df_users[df_users["User"] == user_id]
    user_preference = user_preference.drop(user_preference.columns[[0]], axis = 1)
    movie_preference = df_movie.drop(df_movie.columns[[0,1]], axis = 1)
    user_preference = user_preference.to_numpy()
    movie_preference = movie_preference.to_numpy()

    return user_preference.dot(movie_preference.T)

In [ ]:
#In the movies dataset, if you had to recommend 3 movies to the user with ID=5338, which ones would you choose?

In [67]:
mv =predict_movies(5338)
ss = np.argsort(mv.flatten())[::-1] #Sorting  the userid
df_movie.iloc[ss[0:3], 1]

51     The Matrix Reloaded (2003)
84    The Bourne Supremacy (2004)
83     The Bourne Identity (2002)
Name: Title, dtype: object

In [68]:
#In the movies dataset, if you had to recommend 2 movies to the user with ID=5323, which ones would you choose?
mv =predict_movies(5323)
ss = np.argsort(mv.flatten())[::-1] #Sorting  the userid
df_movie.iloc[ss[0:3], 1]

21    The Dark Knight (2008)
27      The Godfather (1972)
91      The Lion King (1994)
Name: Title, dtype: object

In [47]:
#np.argsort(mv)
#np.sort(mv.flatten())[::-1]


In [44]:
mv[0, 90]

-1.3705332823952836

In [49]:
df_movie.iloc[ss[0:3], 1]

15         The Lord of the Rings: The Two Towers (2002)
16    The Lord of the Rings: The Return of the King ...
14    The Lord of the Rings: The Fellowship of the R...
Name: Title, dtype: object

In [42]:
np.sort(mv.flatten())[::-1]

array([ 1.892e+00,  1.892e+00,  1.859e+00,  1.213e+00,  1.149e+00,
        1.044e+00,  9.952e-01,  7.766e-01,  7.704e-01,  7.003e-01,
        6.680e-01,  5.593e-01,  5.582e-01,  5.000e-01,  4.845e-01,
        4.682e-01,  3.693e-01,  3.523e-01,  3.158e-01,  3.036e-01,
        2.763e-01,  2.718e-01,  2.337e-01,  2.280e-01,  1.752e-01,
        1.642e-01,  1.583e-01,  1.390e-01,  1.353e-01,  1.327e-01,
        1.299e-01,  1.051e-01,  1.035e-01,  1.002e-01,  8.503e-02,
        7.820e-02,  6.003e-02,  5.046e-02,  4.437e-02,  3.095e-03,
        2.190e-04, -1.440e-03, -2.032e-02, -2.876e-02, -5.217e-02,
       -5.467e-02, -6.335e-02, -6.962e-02, -7.422e-02, -7.572e-02,
       -8.839e-02, -9.195e-02, -9.679e-02, -9.778e-02, -1.010e-01,
       -1.052e-01, -1.252e-01, -1.394e-01, -1.475e-01, -1.557e-01,
       -1.625e-01, -1.634e-01, -1.733e-01, -1.877e-01, -1.926e-01,
       -1.971e-01, -1.980e-01, -2.240e-01, -2.262e-01, -2.296e-01,
       -2.373e-01, -2.381e-01, -2.583e-01, -2.647e-01, -3.048e

In [ ]:
predict_rating(14,25)

0.18591442812616604